## ComCam Electrometer files in the LFA
In this notebook, we show how to query the ComCam electrometer\
LFA files.\
Craig Lage - 02-Aug-24

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
from lsst.resources import ResourcePath
from lsst.daf.butler import Butler
from astropy.time import Time, TimeDelta
from lsst_efd_client import EfdClient

In [ ]:
butler = Butler('/repo/embargo', collections=["LSSTComCam/raw/all","LSSTComCam/calib"])
client = EfdClient('usdf_efd')

## First, get a list of exposures

In [ ]:
dayObs = 20240729
instrument = "LSSTComCam"

exposureList = []
for record in butler.registry.queryDimensionRecords("exposure", 
                    where=f"exposure.day_obs={dayObs} and instrument='LSSTComCam'"):
    exposureList.append([record.id, record])
exposureList.sort(key=lambda x: x[0])
for [id,record] in exposureList:
    print(record.id, record.observation_type, record.exposure_time, record.physical_filter)


# Get the LFA file associated with an image

In [ ]:
expIds = []
lenDatas = []
expTimes = []
for expId in range(2024072900178, 2024072900190):
    mData = butler.get('raw.metadata', detector=4, exposure=expId, instrument=instrument)
    start = Time(mData['DATE-END'], scale='tai').utc
    end = start + TimeDelta(8.0, format='sec')
    urls = await client.select_time_series('lsst.sal.Electrometer.logevent_largeFileObjectAvailable', \
                                            ['*'],  start, end, index=101)
    msg = await client.select_time_series('lsst.sal.Electrometer.logevent_logMessage', \
                                            ['message'],  start, end, index=101)
    usdf_locator = urls.iloc[0]['url'].replace('https://s3.cp.lsst.org/', 's3://rubin:')
    print(usdf_locator)
    rp = ResourcePath(usdf_locator)
    with rp.open(mode="rb") as f:
        hdu_list = fits.open(f)
    data = hdu_list[1].data
    expIds.append(expId)
    lenDatas.append(len(data))
    expTimes.append(mData['EXPTIME'])

In [ ]:
for i in range(len(expIds)):
    print(f"expId={expIds[i]}, expTime={expTimes[i]}, length of LFA={(lenDatas[i]*0.1):.1f}")

In [ ]:
seqNums = []
intensities = []
for expId in range(2024072900178, 2024072900378):
    mData = butler.get('raw.metadata', detector=4, exposure=expId, instrument=instrument)
    start = Time(mData['DATE-END'], scale='tai').utc
    end = start + TimeDelta(8.0, format='sec')
    urls = await client.select_time_series('lsst.sal.Electrometer.logevent_largeFileObjectAvailable', \
                                            ['*'],  start, end, index=101)
    usdf_locator = urls.iloc[0]['url'].replace('https://s3.cp.lsst.org/', 's3://rubin:')
    rp = ResourcePath(usdf_locator)
    with rp.open(mode="rb") as f:
        hdu_list = fits.open(f)
    data = hdu_list[1].data
    nMeas = 0
    dataSum = 0
    for i in range(len(data)):
        nMeas += 1
        dataSum += -data[i][0]
    mean = dataSum / nMeas
    seqNums.append(expId - 2024072900000)
    intensities.append(mean)

In [ ]:
plt.title("Photodiode variation during 2024-07-29 ComCam calibration run")
plt.plot(seqNums, np.array(intensities) * 1.0E9)
minMaxPct = (np.max(intensities) - np.min(intensities)) / np.mean(intensities) * 100.0
plt.xlabel("seqNum")
plt.ylabel("Mean photodiode Current (nA)")
plt.text(275, 9.7, f"(Max - Min) / Mean = {minMaxPct:.2f} %")
plt.savefig(f"/home/c/cslage/u/ComCam/images/ComCam_Photodiode_Variation_29Jul24.png")

In [ ]:
print(start)
for i in range(len(msg)):
    if "Scan Summary of Signal" in msg.iloc[i]['message']:
        print(msg.index[i])
        print(msg.iloc[i]['message'])

In [ ]:
for expId in [2024072900024, 2024072900178]:
    print(f"expId={expId}")
    mData = butler.get('raw.metadata', detector=4, exposure=expId, instrument=instrument)
    date_beg = Time(mData['DATE-BEG'], scale='tai').utc
    print(f"DATE_BEG (in UTC) = {date_beg}")
    date_end = Time(mData['DATE-END'], scale='tai').utc
    print(f"DATE_END (in UTC) = {date_end}")
    
    start = Time(mData['DATE-END'], scale='tai').utc
    end = start + TimeDelta(8.0, format='sec')
    urls = await client.select_time_series('lsst.sal.Electrometer.logevent_largeFileObjectAvailable', \
                                        ['*'],  start, end, index=101)
    try:
        print(f"URL index (in UTC) = {urls.index[0]}")
    except:
        continue
    msg = await client.select_time_series('lsst.sal.Electrometer.logevent_logMessage', \
                                            ['message'],  start, end, index=101)
    for i in range(len(msg)):
        if "Scan Summary of Signal" in msg.iloc[i]['message']:
            print(f"Message index (in UTC) = {msg.index[i]}")
            print(msg.iloc[i]['message'])
    print()